In [12]:
SOURCE_MARKDOWN = """
**Subject: The Wordpress vs WP Engine Drama**

In a surprising twist of events, the open-source giant WordPress is entangled in a dramatic controversy surrounding its licensing policies aimed squarely at WP Engine, stirring fierce discussions across the web. **"How can you expect any goodwill towards open-source from a community when you insist on enforcing fees?"** This quote captures the mounting frustration many in the tech community feel as these events unfold.

**Background Context**  
WordPress, an open-source content management system, has thrived on its community-driven model for years. However, recently, the organization faced backlash due to new licensing fee structures selectively impacting WP Engine, a leading managed Wordpress host. This move has raised concerns among other hosting companies and developers about potential instability and implications for open-source projects.

---

**The Emergence of Tension**  
The friction began to surface as Automattic, the parent company of WordPress, targeted WP Engine for allegedly not contributing enough to the WordPress ecosystem. **James Ivings** pointed out in one of his tweets, *“You can’t go IPO with just a happy customer base, you need to be extracting profits from the entire market (via licensing).”* This sentiment echoes the belief that the motives behind the licensing changes extend beyond simply supporting the open-source initiative. ([source](https://twitter.com/jamesivings/status/1839422193681481750))

---

**Mixed Reactions Among Developers**  
As the news broke, reaction from the community was polarized. While some developers supported Automattic's actions as a means to ensure sustainable growth, others voiced concerns about stifling innovation and creating a hostile environment for open source. One developer remarked that experienced consequences, like **“Being blocked from installing plugins,”** could negatively impact WP Engine's customer base and, subsequently, WordPress's reputation as a stable and reliable platform. ([source](https://twitter.com/arvidkahl/status/1839445536686174387))

---

**The Financial Perspective**  
Conversations centered around revenue models emerged, as seen in **Danny Postmaa's** reflections on his experiences, *"more growth != more support tickets."* He noted that despite their expanding user base, support requests remained steady, indicating a deeper complexity in managing resources amid growth. ([source](https://twitter.com/dannypostmaa/status/1839847665338925293))

There’s a growing belief that this drama could lead to significant changes in how hosting services operate with WordPress, resulting in a shift towards licensing strategies that others, like NewFoldDigital, have already embraced. Many felt securing licensing is a smart strategic move, potentially raising the licensing fees for all WP hosting partners. ([source](https://twitter.com/jessethanley/status/1839569215000588641))

---

**Community Support and Reactions**  
While the turmoil has pushed various developers to share their thoughts online, there’s a palpable sense of disbelief among users and developers alike. The unforeseen changes raise questions about open-source integrity, As **Arvid Kahl** noted, emphasizing the need for transparency from Automattic in their reasoning. *“I hope the ecosystem is self-healing. I just hope WPE being blocked from installing can be healed without causing massive reputational damage,”* he expressed. ([source](https://twitter.com/arvidkahl/status/1839445536686174387))

---

In conclusion, the **WordPress vs. WP Engine drama** reveals not just a licensing issue but digs into the larger questions surrounding the sustainability of open-source models and the balance of growth, community trust, and corporate influence. As this saga unfolds, the tech community watches closely, ready to respond.
"""


In [13]:
from typing import Literal, Optional, List, Union
from pydantic import BaseModel, Field

STORYBOARD_PROMPT = """
I will provide you with a source string that represents a news article.

Your job is to create a storyboard for a video that will be generated from the news article.

The storyboard should be an array of objects, where each object represents a starting frame for a video, which will then be animated to create a video segment with a fixed duration.

Don't put two of the same type in a row, and use at least one of each type.

I will also supply a total duration for the video, generate enough frames to fill the duration given a duration of 2 seconds for each frame. Print out the total number of frames you will generate.
"""


TYPE_DESCRIPTION = {
    "meme": "a meme image, that will be animated",
    "twitter_screenshot": "a screenshot of a twitter thread",
    "stock_video": "a stock video based on an initial image",
}


STOCK_IMAGE_DESCRIPTION = """
A description of an image that will be used as a prompt to generate a stock image. We want the image to be photo realistic, showing a scene that matches the source markdown in some way.
"""


class StoryboardItem(BaseModel):
    type: Literal["meme", "twitter_screenshot", "stock_video"] = Field(
        ..., description=str(TYPE_DESCRIPTION)
    )
    stock_image_description: Union[str, None] = Field(
        ..., description=STOCK_IMAGE_DESCRIPTION
    )
    twitter_url: Union[str, None] = Field(
        ...,
        description="The url of the twitter thread to screenshot if the type is twitter_screenshot. Leave blank if the type is not twitter_screenshot. Make sure the tweet is in the source markdown.",
    )


class Storyboard(BaseModel):
    items: List[StoryboardItem] = Field(..., description="The storyboard items")
    total_duration: int = Field(
        ..., description="The total duration of the video in seconds"
    )
    total_frames: int = Field(
        ..., description="The total number of frames in the video"
    )


In [14]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
openai_client = OpenAI()

TOTAL_DURATION = 40

# Call OpenAI API with GPT-4 Turbo
response = openai_client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": STORYBOARD_PROMPT},
        {"role": "user", "content": SOURCE_MARKDOWN},
        {
            "role": "user",
            "content": f"The total duration of the video is {TOTAL_DURATION} seconds.",
        },
    ],
    response_format=Storyboard,
)

# Extract the generated storyboard
storyboard = response.choices[0].message.parsed


print("Storyboard: ")
print("Duration: ", storyboard.total_duration)
print("Total Frames: ", storyboard.total_frames)
for item in storyboard.items:
    print(item)


Storyboard: 
Duration:  40
Total Frames:  20
type='meme' stock_image_description='A dramatic courtroom scene with WordPress and WP Engine logos facing off, symbolizing the controversy.' twitter_url=None
type='twitter_screenshot' stock_image_description=None twitter_url='https://twitter.com/jamesivings/status/1839422193681481750'
type='stock_video' stock_image_description='Developers at computers looking frustrated and discussing licensing fees, symbolizing mixed reactions.' twitter_url=None
type='meme' stock_image_description="Two developers with speech bubbles, one saying 'more growth != more support tickets,' referencing Danny Postmaa's statement." twitter_url=None
type='twitter_screenshot' stock_image_description=None twitter_url='https://twitter.com/arvidkahl/status/1839445536686174387'
type='stock_video' stock_image_description='A meeting of web hosting company executives, pondering over licensing strategies and revenue models.' twitter_url=None
type='meme' stock_image_description

In [ ]:
%reload_ext autoreload
%autoreload 2


import aiohttp


IDEOGRAM_URL = "https://api.ideogram.ai/generate"

IDEOGRAM_HEADERS = {
    "Api-Key": os.getenv("IDEOGRAM_API_KEY"),
    "Content-Type": "application/json",
}


async def generate_ideo_image(prompt: str, starting_image_url: Optional[str] = None):
    image_request = {
        "image_request": {
            "prompt": prompt,
            "model": "V_2",
            "magic_prompt_option": "AUTO",
            "aspect_ratio": "ASPECT_9_16"
        }
    }

    if starting_image_url:
        image_request["image_request"]["keyframe"] = {
            "frame0": {"type": "image", "url": starting_image_url}
        }

    async with aiohttp.ClientSession() as session:
        async with session.post(
            IDEOGRAM_URL, json=image_request, headers=IDEOGRAM_HEADERS
        ) as response:
            return await response.json()


In [20]:
import boto3
from botocore.client import Config

ACCOUNT_ID = os.environ.get('R2_ACCOUNT_ID')
ACCESS_KEY_ID = os.environ.get('R2_ACCESS_KEY_ID') 
SECRET_ACCESS_KEY = os.environ.get('R2_SECRET_ACCESS_KEY')
BUCKET_NAME = os.environ.get('R2_BUCKET_NAME')
CLOUDFLARE_BUCKET_PUBLIC_URL = "https://pub-2576bbab2f764a5a9c3fdc59f470ef1a.r2.dev"

s3_client = boto3.client('s3',
    endpoint_url=f'https://{ACCOUNT_ID}.r2.cloudflarestorage.com',
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY,
    config=Config(signature_version='s3v4')
)

async def upload_to_cloudflare(image_path):
    try:
        with open(image_path, 'rb') as file:
            object_name = os.path.basename(image_path)
            s3_client.upload_fileobj(file, BUCKET_NAME, object_name)

            file_name = image_path.split('/')[-1]
            cloudflare_url = f"{CLOUDFLARE_BUCKET_PUBLIC_URL}/{file_name}"
            return cloudflare_url
    except Exception as e:
        print(f"Error uploading to Cloudflare R2: {str(e)}")
        return None


In [21]:
from tweetcapture import TweetCapture
import asyncio
import os
from PIL import Image


async def capture_tweet(url, port):
    try:
        tweet = TweetCapture()
        tweet.add_chrome_argument(f"--remote-debugging-port={port}")
        
        # Create 'tweets' directory if it doesn't exist
        os.makedirs('tweets', exist_ok=True)
        
        # Generate a filename based on the URL and username
        username = url.split('/')[-3]  # Assuming the URL format is twitter.com/username/status/id
        filename = f"tweets/{username}_{url.split('/')[-1]}.png"
        
        await tweet.screenshot(url, path=filename, overwrite=True)

        return filename
    except Exception as e:
        print(f"Error capturing tweet {url}: {str(e)}")
        return None

def create_backdrop(tweet_image_path, output_path):
    tweet_image = Image.open(tweet_image_path)
    tweet_width, tweet_height = tweet_image.size

    # Add padding
    padding = 50
    tweet_width_padded = tweet_width + 2 * padding
    tweet_height_padded = tweet_height + 2 * padding

    # Create a 9:16 backdrop
    backdrop_width = max(tweet_width_padded, int(tweet_height_padded * 9 / 16))
    backdrop_height = max(tweet_height_padded, int(tweet_width_padded * 16 / 9))
    
    # Determine background color based on tweet image
    tweet_colors = tweet_image.getcolors(tweet_image.size[0] * tweet_image.size[1])
    avg_color = sum(color[0] * color[1][0] for color in tweet_colors) / sum(color[0] for color in tweet_colors)
    bg_color = 'white' if avg_color < 128 else 'black'
    
    backdrop = Image.new('RGB', (backdrop_width, backdrop_height), bg_color)
    
    # Calculate position to center the tweet
    x = (backdrop_width - tweet_width_padded) // 2
    y = (backdrop_height - tweet_height_padded) // 2
    
    # Create a new image with padding for shadow
    padded_tweet = Image.new('RGBA', (tweet_width_padded, tweet_height_padded), (0, 0, 0, 0))
    
    
    # Paste the original tweet image
    padded_tweet.paste(tweet_image, (padding, padding))
    
    # Paste the padded tweet onto the backdrop
    backdrop.paste(padded_tweet, (x, y), padded_tweet)
    
    backdrop.save(output_path)
    return output_path

async def capture_tweets(tweet_urls):
    port = 9222
    tasks = []
    for url in tweet_urls:
        tasks.append(asyncio.create_task(capture_tweet(url, port)))
        port += 1

    filenames = []
    for task in asyncio.as_completed(tasks):
        try:
            filename = await task
            if filename:
                output_filename = f"{os.path.splitext(filename)[0]}_backdrop.png"
                backdrop_filename = create_backdrop(filename, output_filename)
                url = await upload_to_cloudflare(backdrop_filename)
                filenames.append(url)
        except Exception as e:
            print(f"Error processing task: {str(e)}")

    return filenames


In [22]:
from lumaai import AsyncLumaAI
import time
import json
from typing import Optional

MAX_ATTEMPTS = 30
POLL_INTERVAL = 5

client = AsyncLumaAI()

async def generate_luma_video(prompt: Optional[str] = None, start_image_url: Optional[str] = None, aspect_ratio: str = "16:9" ):
    generation = await client.generations.create(
        prompt=prompt,
        keyframes={
            "frame0": {
                "type": "image",
                "url": start_image_url
            }
        } if start_image_url else {},
        aspect_ratio=aspect_ratio
    )
    return generation.id

async def poll_generation(generation_id, max_attempts=MAX_ATTEMPTS, delay=POLL_INTERVAL):
    for attempt in range(max_attempts):
        print(f"Attempt {attempt + 1}/{max_attempts} to poll generation {generation_id}")
        status = await client.generations.get(generation_id)
        print(f"Current status: {status.state}")
        if status.state == "completed":
            print(f"Generation {generation_id} completed successfully")
            return status
        elif status.state == "failed":
            print(f"Generation {generation_id} failed")
            raise Exception(f"Generation failed: {status.failure_reason}")
        
        print(f"Waiting {delay} seconds before next attempt")
        time.sleep(delay)

    print(f"Max attempts ({max_attempts}) reached for generation {generation_id}")
    raise Exception("Max attempts reached")

In [23]:
import requests
from io import BytesIO
import numpy as np
import os

def create_meme_backdrop(meme_url, backdrop_width=1080, backdrop_height=1920):
    # Download the meme image
    response = requests.get(meme_url)
    meme_image = Image.open(BytesIO(response.content))

    # Create a 9:16 aspect ratio backdrop
    img_array = np.array(meme_image)
    average_color = np.mean(img_array)
    background_color = (0, 0, 0) if average_color > 128 else (255, 255, 255)
    
    backdrop = Image.new('RGB', (backdrop_width, backdrop_height), background_color)

    # Calculate the maximum size for the meme image to fit the backdrop
    meme_aspect_ratio = meme_image.width / meme_image.height
    backdrop_aspect_ratio = backdrop_width / backdrop_height

    if meme_aspect_ratio > backdrop_aspect_ratio:
        # Meme is wider, fit to width
        new_width = backdrop_width
        new_height = int(backdrop_width / meme_aspect_ratio)
    else:
        # Meme is taller, fit to height
        new_height = backdrop_height
        new_width = int(backdrop_height * meme_aspect_ratio)

    # Resize meme image to fit within the backdrop while maintaining aspect ratio
    meme_image = meme_image.resize((new_width, new_height), Image.LANCZOS)

    # Calculate position to center the meme image
    x = (backdrop_width - new_width) // 2
    y = (backdrop_height - new_height) // 2

    # Paste the meme image onto the backdrop
    backdrop.paste(meme_image, (x, y))

    # Save the combined image to a BytesIO object
    combined_image_io = BytesIO()
    backdrop.save(combined_image_io, format='PNG')
    combined_image_io.seek(0)

    # Save the combined image to the memes directory
    
    # Ensure the memes directory exists
    memes_dir = 'memes'
    os.makedirs(memes_dir, exist_ok=True)
    
    # Generate a unique filename
    import uuid
    filename = f'meme_{uuid.uuid4()}.png'
    filepath = os.path.join(memes_dir, filename)
    
    # Save the image
    with open(filepath, 'wb') as f:
        f.write(combined_image_io.getvalue())
    
    print(f"Meme saved to: {filepath}")

    return filepath 



In [24]:
import os
from pyairtable import Api
from dotenv import load_dotenv
from pydantic import BaseModel
load_dotenv()


api = Api(os.environ['AIRTABLE_API_KEY'])
table = api.table('appi0R6F1ckhy8JpZ', 'table1')


# Reformat the data
records = table.all()

# Reformat the data
reformatted_data = {}

for record in records:
    name = record['fields'].get('Name', '')
    notes = record['fields'].get('Notes', '')
    image_url = ''
    
    # Get the first image URL if available
    if 'image' in record['fields'] and record['fields']['image']:
        image_url = record['fields']['image'][0].get('url', '')
    
    reformatted_data[name] = {
        'notes': notes,
        'image_url': image_url
    }



class ImageUrl(BaseModel):
    url: str


async def process_video(item):
    if item.type == "twitter_screenshot":
        raise ValueError("Twitter screenshots dont make videos")
    elif item.type == "stock_video":
        ideogram_response = await generate_ideo_image(item.stock_image_description)
        luma_video_id = await generate_luma_video(prompt=None, 
                                            start_image_url=ideogram_response['data'][0]['url'])
        
    elif item.type == "meme":
        dict_memes = json.dumps(reformatted_data)
        model_response = openai_client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content": "You are a helpful assistant"},
                {"role": "user", "content": f"Given the following memes {dict_memes}, give the closest matching meme URL to {item.stock_image_description}"},
            ],
            response_format=ImageUrl,
        )
        meme_url = model_response.choices[0].message.parsed

        print("meme_url", meme_url)

        meme_backdrop = create_meme_backdrop(meme_url.url)

        url = await upload_to_cloudflare(meme_backdrop)


        luma_video_id = await generate_luma_video(prompt=None, start_image_url=url)
    result = await poll_generation(luma_video_id)
    return result






In [25]:
print(storyboard.items[0])
await process_video(storyboard.items[0])


type='meme' stock_image_description='A dramatic courtroom scene with WordPress and WP Engine logos facing off, symbolizing the controversy.' twitter_url=None
meme_url url='https://v5.airtableusercontent.com/v3/u/33/33/1727575200000/ydC_ayEaGWeOIErF1fQqDw/kWG_RaLQe_7t9aVI4M5MqRVfBXSp-DOZsRkVRDd6s7nRfzqXonvJTe9L3Fz6RwagwnlmMn2ed5aGutWNMPvLD4a_fmsGaNPfE5yFgHN38_JvTTIMuVDWclsIgY382h9C-ayTldhJRYj1mN2QVQEylM_5Cl1q4hN8XNCOwbpRq4o/pg2YaWkYziW03lqa9aV9lXEXWwtd5cLUO_hj1paMV5I'
Meme saved to: memes/meme_4e6323df-ca1a-47a2-8839-df0da98bcec9.png
Attempt 1/30 to poll generation 65b2471a-3024-426e-ab5b-f047b999bbd1
Current status: dreaming
Waiting 5 seconds before next attempt
Attempt 2/30 to poll generation 65b2471a-3024-426e-ab5b-f047b999bbd1
Current status: dreaming
Waiting 5 seconds before next attempt
Attempt 3/30 to poll generation 65b2471a-3024-426e-ab5b-f047b999bbd1
Current status: dreaming
Waiting 5 seconds before next attempt
Attempt 4/30 to poll generation 65b2471a-3024-426e-ab5b-f047b999b

Generation(id='65b2471a-3024-426e-ab5b-f047b999bbd1', assets=Assets(video='https://storage.cdn-luma.com/lit_lite_inference_v1.6-xl/a13e8dd2-c70e-4368-a69e-3b52e7935e79/75dc92f8-0e72-4465-9081-bfb0da4e2622_video0e55a90eb4aff4c408d4cbdb72c81c37e.mp4'), created_at=datetime.datetime(2024, 9, 28, 22, 45, 32, 717000, tzinfo=TzInfo(UTC)), failure_reason=None, request=Request(aspect_ratio='16:9', keyframes=RequestKeyframes(frame0=RequestKeyframesFrame0ImageReference(type='image', url='https://storage.cdn-luma.com/app_data/photon/user_uploads/8a41abf8-219c-481f-99ac-4bdc4d0da816/77e2f677-b5a2-431d-9381-220a071c887f_image.jpg'), frame1=None), loop=False, prompt=''), state='completed', version=None)

In [ ]:

async def fetch_all_resources(processed_items):
    tweet_urls = [item.twitter_url for item in processed_items if item.type == 'twitter_screenshot']
    image_items = [item for item in processed_items if item.type in ['stock_video', 'meme']]

    tweet_tasks = [capture_tweets(tweet_urls)] if tweet_urls else []
    image_tasks = [process_video(item) for item in image_items]
    results = await asyncio.gather(*tweet_tasks, *image_tasks)

    tweet_files = results[0] if tweet_urls else []
    video_results = results[1:] if image_items else []

    return tweet_files, video_results

async def main():
    tweet_files, video_results = await fetch_all_resources(storyboard.items)


    combined_resources = []

    tweet_index = 0
    image_index = 0

    for item in storyboard.items:
        if item.type == 'twitter_screenshot' :

            tweet_id = item.twitter_url.split("/")[-1]
  
            if any(tweet_id in tweet_file for tweet_file in tweet_files):
                url = tweet_files[tweet_index]
               
                combined_resources.append({
                    "type": "twitter_screenshot",
                    "url": url
                })
                tweet_index += 1
        elif item.type in ['stock_video', 'meme']:
            combined_resources.append({
                "type": item.type,
                "result": video_results[image_index]
            })
            image_index += 1

    print("Combined resources in storyboard order:")
    print(combined_resources)
    return combined_resources

combined_resources = await main()

In [ ]:
from IPython.display import Image

for resource in combined_resources:
    if resource["type"] == "twitter_screenshot":
        print(resource["url"])
        display(Image(url=resource["url"]))
    elif resource["type"] == "stock_video" or resource["type"] == "meme":
        print(resource["result"].assets.video)

